In [1]:
# Import live code changes in
%load_ext autoreload
%autoreload 2

import os
from pathlib import Path
import pandas as pd
import numpy as np
from sovereign.flood import simple_risk_overlay, flopros_risk_overlay, make_uncertainty_curve
from sovereign.macroeconomic import prepare_DIGNAD, run_DIGNAD
from sovereign.utils import sum_rasters, raster_sum

#### Set filepaths and provide data info

In [15]:
# Set filepaths
root = Path.cwd().parent # find project root
flood_path = os.path.join(root, 'inputs', 'flood', 'validation', 'THA_2011_flood.tif')
exposure_dir = os.path.join(root, 'outputs', 'exposure')
vulnerability_path = os.path.join(root, 'inputs', 'flood', 'vulnerability', 'jrc_depth_damage.csv')
# Capital stock paths
priv_inf = os.path.join(exposure_dir, 'inf_priv_capstock.tif')
priv_res = os.path.join(exposure_dir, 'res_priv_capstock.tif')
priv_nres = os.path.join(exposure_dir, 'nres_priv_capstock.tif')
pub_inf = os.path.join(exposure_dir, 'inf_pub_capstock.tif')
pub_nres = os.path.join(exposure_dir, 'nres_pub_capstock.tif')
# GVA paths
agr_gva = os.path.join(exposure_dir, 'THA_agr_GVA.tif')
man_gva = os.path.join(exposure_dir, 'THA_man_GVA.tif')
ser_gva = os.path.join(exposure_dir, 'THA_ser_GVA.tif')
# Flood validation risk map folder
risk_results = os.path.join(root, 'outputs', 'flood', 'validation', 'maps')
# DIGNAD 2022 Calibration
THA_calibration_path = os.path.join(root, "inputs", "macro", "THA_2022_calibration_final.csv")

In [3]:
# Extract vulnerability curves
vuln_df = pd.read_csv(vulnerability_path)
v_heights = vuln_df['flood_depth'].to_list()
v_res = vuln_df['asia_residential'].to_list()
v_com = vuln_df['asia_commercial'].to_list()
v_ind = vuln_df['asia_industrial'].to_list()
v_agr = vuln_df['asia_agriculture'].to_list()
v_inf = vuln_df['asia_infrastructure'].to_list()

#### Run Risk Analysis

In [5]:
# Set filepaths
# Check to see if results directory exists
if not os.path.exists(risk_results):
    os.makedirs(risk_results)
priv_inf_risk = os.path.join(risk_results, f'THA_2011_priv_inf_cap_damages.tif')
priv_res_risk = os.path.join(risk_results, f'THA_2011_priv_res_cap_damages.tif')
priv_nres_risk = os.path.join(risk_results, f'THA_2011_priv_nres_cap_damages.tif')
pub_inf_risk = os.path.join(risk_results, f'THA_2011_pub_inf_cap_damages.tif')
pub_nres_risk = os.path.join(risk_results, f'THA_2011_pub_nres_cap_damages.tif')
agr_gva_risk = os.path.join(risk_results, f'THA_2011_agr_gva_losses.tif')
man_gva_risk = os.path.join(risk_results, f'THA_2011_man_gva_losses.tif')
ser_gva_risk = os.path.join(risk_results, f'THA_2011_ser_gva_losses.tif')

In [5]:
# Run overlay analysis (skip if already done)
simple_risk_overlay(flood_path, priv_inf, priv_inf_risk, [v_heights, v_inf])
simple_risk_overlay(flood_path, priv_res, priv_res_risk, [v_heights, v_res])
simple_risk_overlay(flood_path, priv_nres, priv_nres_risk, [v_heights, v_com])
simple_risk_overlay(flood_path, pub_inf, pub_inf_risk, [v_heights, v_inf])
simple_risk_overlay(flood_path, pub_nres, pub_nres_risk, [v_heights, v_com])
simple_risk_overlay(flood_path, agr_gva, agr_gva_risk, [v_heights, v_agr])
simple_risk_overlay(flood_path, man_gva, man_gva_risk, [v_heights, v_ind])
simple_risk_overlay(flood_path, ser_gva, ser_gva_risk, [v_heights, v_com])
# We are the going to sum public and private capital stock into individual risk maps 
sum_rasters([priv_inf_risk, priv_res_risk, priv_nres_risk], os.path.join(risk_results, f'THA_2011_priv_cap_damages.tif'))
sum_rasters([pub_inf_risk, pub_nres_risk], os.path.join(risk_results, f'THA_2011_pub_cap_damages.tif'))

NameError: name 'sum_rasters' is not defined

#### Sum National Numbers

In [11]:
pub_cap = raster_sum(os.path.join(risk_results, f'THA_2011_pub_cap_damages.tif'))
pri_cap = raster_sum(os.path.join(risk_results, f'THA_2011_priv_cap_damages.tif'))
agr_gva = raster_sum(agr_gva_risk)
man_gva = raster_sum(man_gva_risk)
ser_gva = raster_sum(ser_gva_risk)
print('Total Capital Stock Damages:', pub_cap + pri_cap)
print('Total GVA Losses:', agr_gva + man_gva + ser_gva)

Total Capital Stock Damages: 31790117000.0
Total GVA Losses: 8345173000.0


In [32]:
pri_cap

21933232000.0

#### DIGNAD Simulation

In [25]:
# MACROECONOMIC MODELLING PARAMETERS
# Flood Risk Number Transformation Parameters
Thai_GDP = 496e9 # 2022 numbers in USD
# National GVA figures from DOSE
agr_GVA_tot = 42880325598
man_GVA_tot = 162659433017
ser_GVA_tot = 316647741231
TRADABLE_SHARES = {
    "Agriculture": 1.0,
    "Manufacturing": 0.7,
    "Service": 0.5,
}
# DIGNAD Parameters
sim_start_year = 2022
nat_disaster_year = 2023
recovery_period = 1 # years
reconstruction_efficiency = 0 # non-adjustable parameter
public_debt_premium = 0 # non-adjustable parameter

In [14]:
# Prepare disaster inputs for DIGNAD
tradable_GVA = ((agr_GVA_tot*TRADABLE_SHARES['Agriculture'])+(man_GVA_tot*TRADABLE_SHARES['Manufacturing'])+(ser_GVA_tot*TRADABLE_SHARES['Service']))
nontradable_GVA = ((agr_GVA_tot*(1-TRADABLE_SHARES['Agriculture']))+(man_GVA_tot*(1-TRADABLE_SHARES['Manufacturing']))+(ser_GVA_tot*(1-TRADABLE_SHARES['Service'])))
dY_T = ((agr_gva*TRADABLE_SHARES['Agriculture'])+(man_gva*TRADABLE_SHARES['Manufacturing'])+(ser_gva*TRADABLE_SHARES['Service']))/tradable_GVA
dY_N = ((agr_gva*(1-TRADABLE_SHARES['Agriculture']))+(man_gva*(1-TRADABLE_SHARES['Manufacturing']))+(ser_gva*(1-TRADABLE_SHARES['Service'])))/nontradable_GVA
dK_pri = pri_cap / Thai_GDP
dK_pub = pub_cap / Thai_GDP

In [16]:
# Calibrate DIGNAD
prepare_DIGNAD(THA_calibration_path, adaptation_cost=0)

In [26]:
# Run DIGNAD
gdp_impact, years = run_DIGNAD(sim_start_year, nat_disaster_year, recovery_period, dY_T, dY_N, reconstruction_efficiency,
                                  public_debt_premium, dK_pub, dK_pri, 0.5)

In [31]:
np.average(gdp_impact[1:6])

-0.6414513622718854

In [33]:
gdp_impact

[0.0,
 -1.1295069742930597,
 -0.7643770672723482,
 -0.44862191120099704,
 -0.4437682062969128,
 -0.4209826522961091,
 -0.407824778307031,
 -0.39132825804883975,
 -0.37744523891976023,
 -0.3633317180673945,
 -0.3502494746327134,
 -0.3375531332616699,
 -0.32549108352969425,
 -0.31389950774499686,
 -0.3028106604195946,
 -0.29216887500861377,
 -0.2819626320230806,
 -0.27216265650753524,
 -0.26275051725778154,
 -0.25370504267460126]